<a href="https://colab.research.google.com/github/danort92/Programming-with-Python/blob/main/Python_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import os
from datetime import datetime

#funzione che stampa un messaggio di aiuto con informazioni sui possibili comandi da lanciare
def help_cmd():
    print("\nI comandi disponibili sono i seguenti:\n"+
    "aggiungi: aggiungi un prodotto al magazzino\n"+
    "elenca: elenca i prodotti in magazzino\n"+
    "vendita: registra una vendita effettuata\n"+
    "profitti: mostra i profitti totali\n"+
    "aiuto: mostra i possibili comandi\n"+
    "chiudi: esci dal programma")

    
#####################################################################################################
    
#funzione che, a partire da file .csv con tutti i movimenti di acquisto e vendita,
#genera lo stato del magazzino e verrà richiamata nell funzioni relative all'aggiunta e alla vendita di prodotti
def stock_info():
    with open("stock.csv","r") as csv_file:        
        csv_reader=csv.DictReader(csv_file)
        #leggo le righe del file stock.csv, che si è creato con l'acquisto del primo prodotto e la sua aggiunta al magazzino.
        #genero lista di dizionari: ogni dizionario corrisponde a riga di acquisto o vendita
        list_dict=[]
        for row in list(csv_reader):
            list_dict.append(row)
            
        #modifico la lista appena creata:
        #all'i-esimo dizionario, controllo se il prodotto compare nei precedenti dizionari.
        #se è presente, somma la quantità dell'i-esimo dizionario al dizionario corrispondente alla prima occorrenza dello specifico prodotto (sicuramente corrispodnente ad un acquisto)
        #la lista di dizionari così aggiornata presenta il corretto dato "quantità" all'interno del dizionario
        #corrispondente alla prima occorrenza registrata per ogni prodotto.
        #(se il dizionario corrisponde ad un acquisto avrà sommato un valore positivo, se corrisponde ad una vendita un valore negativo)
        for i,line in enumerate(list_dict):
            prev_lines=[]
            for j in range(i):
                prev_lines.append(list_dict[j]["PRODOTTO"])
            for k,prev_line in enumerate(prev_lines):
                if line["PRODOTTO"]==prev_line:
                    list_dict[k]["QUANTITA'"]=int(list_dict[k]["QUANTITA'"])+int(line["QUANTITA'"])
                else:
                    continue
        
        #compilo variabile tmp, in questo caso il magazzino vero e proprio, che mostra quanti prodotti sono disponibili
        #considerando gli acquisti fatti e le vendite completate
        #nella lista di dizionari precedentemente generata, solo il dizionario corrispondente 
        #alla prima occorrenza di uno specifico prodotto contiene le giuste informazioni sulla quantità:
        #con questo blocco di codice tolgo i dizionari corrispondenti alle occorrenze di uno specifico prodotto dopo la prima,
        #mgenerando quindi una lista i cui dizionari sono unici per prodotto: il magazzino.
        tmp={}           
        for i,row in enumerate(list_dict):
            prev_lines=[]
            for j in range(i):
                prev_lines.append(list_dict[j]["PRODOTTO"])
    
            if not row["PRODOTTO"] in prev_lines:

                info={}
                info["Quantità"]=int(row["QUANTITA'"])
                info["Prezzo di acquisto"]=float(row["PREZZO DI ACQUISTO"])
                info["Prezzo di vendita"]=float(row["PREZZO DI VENDITA"])
                tmp[row["PRODOTTO"]]=info
            else:
                continue
                
        #la funzione ritorna la variabile stock  
        #(dizionario che ha come chiave il prodotto X e come valore un dizionario contenente le info su: 
        #quantità, prezzo di acquisto e prezzo di vendita)
        return tmp 
    
    
 #####################################################################################################   

#funzione che gestisce l'aggiunta dei prodotti nel magazzino
#riceve in input i valori product_name (nome del prodotto) e quantity (quantità)
def add(product_name,quantity):
    
    #creo file stock.csv in append. E' un vero e proprio registro attività: la stessa verrà aggiunta come nuova riga
    with open("stock.csv","a+",newline="") as csv_file:
        columns=["PRODOTTO","QUANTITA'","PREZZO DI ACQUISTO","PREZZO DI VENDITA","OPERAZIONE","DATA E ORA"]
        csv_reader=csv.DictReader(csv_file)
        csv_writer=csv.DictWriter(csv_file,fieldnames=columns)
        
          
        #controllo se il file è vuoto, se lo è scrivo nel file l'headline, contenente, oltre a
        #nome del prodotto - quantità - prezzo di acquisto - prezzo di vendita,
        #anche il tipo di operazione (acquisto/vendita) e l'orario di registrazione dell'operazione
        fileEmpty = os.stat("stock.csv").st_size == 0
        if fileEmpty:
            csv_writer.writeheader()  
        
        #dal registro stock.csv creo una lista con i nomi dei prodotti
        list_products=[]
        csv_file.seek(0)
        for row in csv_reader:
            list_products.append(row["PRODOTTO"])
            
        #controllo se il prodotto X è già presente nella lista appena creata
        #se non è presente chiedo all'utente di inserire le informazioni relative a prezzo di acuisto e prezzo di vendita
        #controllo che l'utente inserisca correttamente numeri decimali
        if not product_name in set(list_products):
            purchase_price,selling_price=None,None
            while purchase_price==None:
                try:
                    purchase_price=float(input("Prezzo di acquisto: "))
                except ValueError:
                    print("\nQuantità inserita non valida: non inserito numero decimale")
            while selling_price==None:
                try:        
                    selling_price=float(input("Prezzo di vendita: "))
                except ValueError:
                    print("\nQuantità inserita non valida: non inserito numero decimale")
            
            #scrivo una nuova riga con le informazioni inserite nel registro stock.csv
            #aggiungendo come tipo di operazione "acquisto" e la data e ora dell'attività
            csv_writer.writerow({"PRODOTTO":product_name,"QUANTITA'":quantity,"PREZZO DI ACQUISTO":purchase_price,"PREZZO DI VENDITA":selling_price,"OPERAZIONE":"acquisto","DATA E ORA":datetime.strftime(datetime.now(), "%d-%m-%Y %H.%M.%S")})
        
        #se invece il prodotto è già presente nel registro, torno all'inizio del registro 
        #e salvo l'indice del dizionario corrispondente alla prima occorrenza del prodotto stesso
        else:
            csv_file.seek(0)
            list_lines=[]
            for row in list(csv_reader)[1:]:
                list_lines.append(row)
            for i,product in enumerate(list_products):
                if product_name==product:
                    z=i
                    break
                else:
                    continue
            
            #scrivo nuova riga con le informazioni inserite relative al nome del prodotto e alla quantità
            #aggiungendo come tipo di operazione "acquisto" e la data e ora dell'attività
            #e le stesse informazioni su prezzo di acquiato e vendita già precedentemente inserite per lo stesso prodotto
            csv_writer.writerow({"PRODOTTO":product_name,"QUANTITA'":quantity,"PREZZO DI ACQUISTO":list_lines[z]["PREZZO DI ACQUISTO"],"PREZZO DI VENDITA":list_lines[z]["PREZZO DI VENDITA"],"OPERAZIONE":"acquisto","DATA E ORA":datetime.strftime(datetime.now(), "%d-%m-%Y %H.%M.%S")})
        
        #stampo l'attività di acquisto
        print(f"AGGIUNTO {quantity} X {product_name}")
    
    #richiamo la funzione stock_info() che aggiorna il magazzino
    stock_info()
        
#####################################################################################################    

#funzione che elenca lo stato del magazzino
def itemize():
    
    #lancio funzione stock_info() che fotografa la situazione del magazzino a partire dal registro attività stock.csv
    #se non trova nulla significa che il file non è ancora stato creato (è stato lanciato il comando "elenca" prima di registrare il primo acquisto)
    try:
        stock=stock_info()
        head_text=["\nPRODOTTO","QUANTITA'","PREZZO"]
        print(*head_text)
        
        #per ogni prodotto all'interno della lista di chiavi del dizionario stock
        #stampo le seguenti informazioni: nome del prodotto, quantità e prezzo di vendita
        #solo se il prodotto non ha quantità 0 (valore risultante dall'ultima vendita che ha azzerato il prodotto nel magazzino)
        for i in range(len(list(stock.keys()))):
            if list(stock.values())[i]["Quantità"]!=0:
                print(f"{list(stock.keys())[i]} "+
                f"{list(stock.values())[i]['Quantità']} "+
                f"€{list(stock.values())[i]['Prezzo di vendita']:.2f} ")
    except FileNotFoundError:
        print("Il registro acquisti/vendite non esiste. Forse non è ancora stato registrato il primo acquisto?...")
        
        
        
#####################################################################################################


#funzione che gestisce l'aggiunta dei prodotti nel magazzino
#riceve in input i valori product_name (nome del prodotto) e quantity (quantità)
def sell(product_name,quantity):
    
    #apro il registro stock.csv già creato con la precedenete aggiunta dei prodotti acquistati
    with open("stock.csv","a+",newline="") as csv_file:
        columns=["PRODOTTO","QUANTITA'","PREZZO DI ACQUISTO","PREZZO DI VENDITA","OPERAZIONE","DATA E ORA"]
        csv_reader=csv.DictReader(csv_file)
        csv_writer=csv.DictWriter(csv_file,fieldnames=columns)
        fileEmpty = os.stat("stock.csv").st_size == 0
        if fileEmpty:
            csv_writer.writeheader()  
        
        #dal registro stock.csv creo una lista con i nomi dei prodotti
        list_products=[]
        csv_file.seek(0)
        for row in csv_reader:
            list_products.append(row["PRODOTTO"])
        
        #se il prodotto non è presente nella lista dei prodotti creata avviso l'utente che il prodotto non è nel magazzino
        if not product_name in set(list_products):
            print("Prodotto non presente in magazzino")
        
        #se invece il prodotto è già presente nel registro, torno all'inizio del registro 
        #e salvo l'indice del dizionario corrispondente alla prima occorrenza del prodotto stesso
        else:
            csv_file.seek(0)
            csv_reader=csv.DictReader(csv_file)
            list_lines=[]
            for row in list(csv_reader):
                list_lines.append(row)
            for i,product in enumerate(list_products):
                if product_name==product:
                    z=i
                    break
                else:
                    continue
            
            #torno all'inizo del registro stock.csv e creo una lista di dizionari, uno per ogni attività registrata
            csv_file.seek(0) 
            csv_reader=csv.DictReader(csv_file)
            list_dict=[]
            for row in list(csv_reader):
                list_dict.append(row)
            
            #modifico la lista appena creata:
            #all'i-esimo dizionario, controllo se il prodotto compare nei precedenti dizionari.
            #se è presente, somma la quantità dell'i-esimo dizionario al dizionario corrispondente alla prima occorrenza dello specifico prodotto (sicuramente corrispondente ad un acquisto)
            #la lista di dizionari così aggiornata presenta il corretto dato "quantità" all'interno del dizionario
            #corrispondente alla prima occorrenza registrata per ogni prodotto.
            #(se il dizionario corrisponde ad un acquisto avrà sommato un valore positivo, se corrisponde ad una vendita un valore negativo)
            for i,line in enumerate(list_dict):
                prev_lines=[]
                for j in range(i):
                    prev_lines.append(list_dict[j]["PRODOTTO"])
                for k,prev_line in enumerate(prev_lines):
                    if line["PRODOTTO"]==prev_line:
                        list_dict[k]["QUANTITA'"]=int(list_dict[k]["QUANTITA'"])+int(line["QUANTITA'"])
                    else:
                        continue
            
            #compilo variabile tmp, che anche in questo caso corrisponde al magazzino, ma prima di registrare la vendita
            #nella lista di dizionari precedentemente generata, solo il dizionario corrispondente 
            #alla prima occorrenza di uno specifico prodotto contiene le giuste informazioni sulla quantità:
            #con questo blocco di codice tolgo i dizionari corrispondenti alle occorrenze di uno specifico prodotto dopo la prima,
            #mgenerando quindi una lista i cui dizionari sono unici per prodotto: il magazzino.
            tmp={}           
            for i,row in enumerate(list_dict):
                prev_lines=[]
                for j in range(i):
                    prev_lines.append(list_dict[j]["PRODOTTO"])

                if not row["PRODOTTO"] in prev_lines:
                    info={}
                    info["Quantità"]=int(row["QUANTITA'"])
                    info["Prezzo di acquisto"]=float(row["PREZZO DI ACQUISTO"])
                    info["Prezzo di vendita"]=float(row["PREZZO DI VENDITA"])
                    tmp[row["PRODOTTO"]]=info

                else:
                    continue

            #una volta calcolato cosa abbiamo a magazzino, controllo che la quantità inserita per le vendita
            #può essere soddisfatta: abbiamo abbastanza prodotto X in magazzino?
            #se sì, procedo a compilare il registro attività con una nuova riga corrispondente alla vendita (quantità negativa), 
            ##aggiungendo come tipo di operazione "vendita" e la data e ora dell'attività
            #se no, stampo un avviso di mancanza di disponibilità
            if quantity<=int(tmp[product_name]["Quantità"]):
                csv_writer.writerow({"PRODOTTO":product_name,"QUANTITA'":-quantity,"PREZZO DI ACQUISTO":list_lines[z]["PREZZO DI ACQUISTO"],"PREZZO DI VENDITA":list_lines[z]["PREZZO DI VENDITA"],"OPERAZIONE":"vendita","DATA E ORA":datetime.strftime(datetime.now(), "%d-%m-%Y %H.%M.%S")})
                stop=1
            else:
                availability=int(tmp[product_name]["Quantità"])
                print(f"Disponibilità max magazzino per prodotto {product_name}: {availability}")
                stop=0
            
            #se vendita avvenuta con successo ritorno variabile stop inizializzata a 1, vicevera a 0
            return stop
        
    #lancio la funzione stock_info, che aggiorna il magazzino dopo l'eventuale vendita avvenuta
    stock_info()

#####################################################################################################

    
#funzione che stampa profitto lordo e netto (considerando cioè il costo di acwuisto dei prodotti)    
def profit():
    
    #apro in lettura il registro attività stock.csv
    with open("stock.csv") as csv_file:
        csv_reader=csv.DictReader(csv_file)
        list_lines=[]
        buying_cost=0
        gross_profit=0
        for row in list(csv_reader):
            list_lines.append(row)
        for line in list_lines:
            
            #se l'operazione è "acquisto" sommo il prezzo totale (prezzo unitario x quantità) nella variabile buying_cost
            if line["OPERAZIONE"]=="acquisto":
                buying_cost+=float(line["PREZZO DI ACQUISTO"])*int(line["QUANTITA'"])
                
            #se l'operazione è "vendita" sommo il prezzo totale (prezzo unitario x quantità) nella variabile gross profit (tenendo conto del segno)
            if line["OPERAZIONE"]=="vendita":
                gross_profit+=-(float(line["PREZZO DI VENDITA"])*int(line["QUANTITA'"]))
    
    #profitto netto differenza tra i due
    net_profit=gross_profit-buying_cost
    print(f"Profitto: lordo = {gross_profit:.2f}€, netto = {net_profit:.2f}€")
    


#####################################################################################################
#####################################################################################################

cmd = None
while cmd!="chiudi":

#####################################################################################################
    cmd = input("\nInserisci un comando: ")
    
#####################################################################################################   
    
    if cmd=="vendita":
    # registra una vendita
        sold_dict={}
        answer="sì"
        # COACH: bastava convertire answer in minuscolo con answer.lower()
        while answer.lower()!="no":
            is_string,quantity=False,None

        #controllo possibili errori di inserimento da parte dell'utente come nome del prodotto: 
        #se si inserisce erroneamente un numero come nome del prodotto viene generata eccezione
            while is_string==False:
                try:
                    product_name=input("Nome del prodotto: ")  
                    if product_name.isnumeric():
                        is_string==False
                        raise ValueError("\nProdotto inserito non valido: inserito valore numerico.")
                        continue
                    else:
                        is_string=True
                except ValueError as e:
                    print(e)

        #controllo possibili errori di inserimento da parte dell'utente come quantità: 
        #se si inserisce erroneamente una stringa come quantità viene generata eccezione
            while quantity==None:
                try:
                    quantity=int(input("Quantità: "))
                except ValueError:
                    print("\nQuantità inserita non valida: non inserito numero intero")
            
            #assegno a variabile stop il parametro stop ottenuto da funzione sell
            #assegno a variabile stock il parametro tmp ottenutoda funzione stock_info
            stop=sell(product_name,quantity)
            stock=stock_info()
            
            #se stop=1 (vendita avvenuta) registro nome prodotto e quantità in un dizionario
            if stop==1:
                sold_dict[product_name]=quantity
            
            #chiedo se si vuole inserire un altro prodotto 
            #(premendo qualsiasi tasto o combinazione diversi da "NO" si richiede una nuova vendita)
            answer=input("Inserire altro prodotto?: [sì/NO]")
        
        #se il dizionario precedentemente creato non è vuoto significa che la vendita è avvenuta e stampo messaggio
        if len(list(sold_dict.keys()))!=0:
            print("\nVENDITA REGISTRATA: ")
            
        #stampo il totale venduto 
        total_sold=0
        for key in list(sold_dict.keys()):
            print(f"{sold_dict[key]} X {key}: €{stock[key]['Prezzo di vendita']}")
            total_sold+=(stock[key]['Prezzo di vendita']*sold_dict[key])
        print(f"\nTotale: €{total_sold:.2f}")
        
        
        
#####################################################################################################

    elif cmd=="profitti":
    # mostra profitti netti e lordi
        profit()
#####################################################################################################   
    
    elif cmd=="aggiungi":
    # aggiungi un prodotto al magazzino
    
        #controllo possibili errori di inserimento da parte dell'utente come nome del prodotto: 
        #se si inserisce erroneamente un numero come nome del prodotto viene generata eccezione
        is_string,quantity=False,None
        while is_string==False:
            try:
                product_name=input("Nome del prodotto: ")  
                if product_name.isnumeric():
                    is_string==False
                    raise ValueError("\nProdotto inserito non valido: inserito valore numerico.")
                    continue
                else:
                    is_string=True
            except ValueError as e:
                print(e)

        #controllo possibili errori di inserimento da parte dell'utente come quantità: 
        #se si inserisce erroneamente una stringa come quantità viene generata eccezione
        while quantity==None:
            try:
                quantity=int(input("Quantità: "))
            except ValueError:
                print("\nQuantità inserita non valida: non inserito numero intero")

        #richiamo funzione add
        add(product_name,quantity)

#####################################################################################################

    elif cmd=="elenca":
    # elenca tutti i prodotti nel magazzino
        itemize()
#####################################################################################################        

    elif cmd=="aiuto":
    # mostra i possibili comandi
        help_cmd()
#####################################################################################################       
    
    elif cmd=="chiudi":
    # saluta e interrompi il programma
        print("Bye Bye")
        break
    else:
    # comando non valido
    # mostra messaggio di aiuto
        print("\nComando non valido")
        help_cmd()


Inserisci un comando: elenca

PRODOTTO QUANTITA' PREZZO
seitan 17 €6.00 

Inserisci un comando: aggiungi
Nome del prodotto: tofu
Quantità: 3
Prezzo di acquisto: 3
Prezzo di vendita: 5
AGGIUNTO 3 X tofu

Inserisci un comando: elenca

PRODOTTO QUANTITA' PREZZO
seitan 17 €6.00 
tofu 3 €5.00 

Inserisci un comando: vendita
Nome del prodotto: seitan
Quantità: 2
Inserire altro prodotto?: [sì/NO]nO

VENDITA REGISTRATA: 
2 X seitan: €6.0

Totale: €12.00

Inserisci un comando: chiudi
Bye Bye
